In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Attempt to Train Polish dataset (100 samples in directory)

Unsuccesful: stuck on epoch 7

In [7]:
import os

from trainer import Trainer, TrainerArgs

from TTS.config.shared_configs import BaseAudioConfig
from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.configs.tacotron2_config import Tacotron2Config
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.tacotron2 import Tacotron2
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor
from TTS.tts.utils.speakers import SpeakerManager

output_path = "/content/drive/MyDrive/Software_proj/Tacatron/tts_train_dir/"

data_path = "/content/drive/MyDrive/Software_proj/Tacatron/tts_train_dir/"

# Using LJSpeech like dataset processing for the blizzard dataset
dataset_config = BaseDatasetConfig(
    formatter="ljspeech",
    meta_file_train="train.csv",
    path=data_path,
)

audio_config = BaseAudioConfig(
    sample_rate=24000,
    do_trim_silence=True,
    trim_db=60.0,
    signal_norm=False,
    mel_fmin=0.0,
    mel_fmax=11025,
    spec_gain=1.0,
    log_func="np.log",
    ref_level_db=20,
    preemphasis=0.0,
)

# Using the standard Capacitron config
capacitron_config = CapacitronVAEConfig(capacitron_VAE_loss_alpha=1.0, capacitron_capacity=50)

config = Tacotron2Config(
    run_name="Capacitron-Tacotron2",
    audio=audio_config,
    capacitron_vae=capacitron_config,
    use_capacitron_vae=True,
    batch_size=32,  # Tune this to your gpu
    max_audio_len=40 * 24000,  # Tune this to your gpu
    min_audio_len=2 * 24000,
    eval_batch_size=4,
    num_loader_workers=2,
    num_eval_loader_workers=2,
    precompute_num_workers=2,
    run_eval=True,
    test_delay_epochs=6,
    ga_alpha=0.0,
    r=2,
    optimizer="CapacitronOptimizer",
    optimizer_params={"RAdam": {"betas": [0.9, 0.998], "weight_decay": 1e-6}, "SGD": {"lr": 1e-5, "momentum": 0.9}},
    attention_type="dynamic_convolution",
    grad_clip=0.0,  # Important! We overwrite the standard grad_clip with capacitron_grad_clip
    double_decoder_consistency=False,
    epochs=25,
    use_phonemes=True,
    phoneme_language="pl",
    phonemizer="espeak",
    phoneme_cache_path=os.path.join(data_path, "phoneme_cache3"),
    stopnet_pos_weight=15,
    print_step=25,
    print_eval=True,
    mixed_precision=False,
    seq_len_norm=True,
    output_path=output_path,
    datasets=[dataset_config],
    use_speaker_embedding=True,
    lr=1e-3,
    lr_scheduler="StepwiseGradualLR",
    lr_scheduler_params={
        "gradual_learning_rates": [
            [0, 1e-3],
            [2e4, 5e-4],
            [4e5, 3e-4],
            [6e4, 1e-4],
            [8e4, 5e-5],
        ]
    },
    scheduler_after_epoch=False,  # scheduler doesn't work without this flag
    # Need to experiment with these below for capacitron
    loss_masking=False,
    decoder_loss_alpha=1.0,
    postnet_loss_alpha=1.0,
    postnet_diff_spec_alpha=0.0,
    decoder_diff_spec_alpha=0.0,
    decoder_ssim_alpha=0.0,
    postnet_ssim_alpha=0.0,
)

ap = AudioProcessor(**config.audio.to_dict())

tokenizer, config = TTSTokenizer.init_from_config(config)

train_samples, eval_samples = load_tts_samples(dataset_config, eval_split=True)

speaker_manager = SpeakerManager()
speaker_manager.set_ids_from_data(train_samples + eval_samples, parse_key="speaker_name")
config.num_speakers = speaker_manager.num_speakers

model = Tacotron2(config, ap, tokenizer, speaker_manager=speaker_manager)

trainer = Trainer(
    TrainerArgs(),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
    training_assets={"audio_processor": ap},
)

trainer.fit()

 > Setting up Audio Processor...
 | > sample_rate:24000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:11025
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60.0
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 | > Found 100 files in /content/drive/MyDrive/Software_proj/Tacatron/tts_train_dir
 > Init speaker_embedding layer.


 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: False
 | > Precision: float32
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 2
 | > Num. of Torch Threads: 1
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=/content/drive/MyDrive/Software_proj/Tacatron/tts_train_dir/Capacitron-Tacotron2-November-01-2023_05+21PM-0000000

 > Model has 34615866 parameters


 > `speakers.pth` is saved to /content/drive/MyDrive/Software_proj/Tacatron/tts_train_dir/Capacitron-Tacotron2-November-01-2023_05+21PM-0000000/speakers.pth.
 > `speakers_file` is updated in the config.json.



 > EPOCH: 0/25
 --> /content/drive/MyDrive/Software_proj/Tacatron/tts_train_dir/Capacitron-Tacotron2-November-01-2023_05+21PM-0000000

 > TRAINING (2023-11-01 17:21:29) 




> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: pl
		| > phoneme backend: espeak
| > Number of instances : 99
 | > Preprocessing samples
 | > Max text length: 250
 | > Min text length: 115
 | > Avg text length: 183.6161616161616
 | 
 | > Max audio length: 358822.0
 | > Min audio length: 240022.0
 | > Avg audio length: 294283.83838383836
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.


...

 | > Synthesizing test sentences.


 ! Run is kept in /content/drive/MyDrive/Software_proj/Tacatron/tts_train_dir/Capacitron-Tacotron2-November-01-2023_05+21PM-0000000


Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/trainer/trainer.py", line 1808, in fit
    self._fit()
  File "/usr/local/lib/python3.10/dist-packages/trainer/trainer.py", line 1764, in _fit
    self.test_run()
  File "/usr/local/lib/python3.10/dist-packages/trainer/trainer.py", line 1680, in test_run
    test_outputs = self.model.test_run(self.training_assets)
  File "/usr/local/lib/python3.10/dist-packages/TTS/tts/models/base_tacotron.py", line 157, in test_run
    outputs_dict = synthesis(
  File "/usr/local/lib/python3.10/dist-packages/TTS/tts/utils/synthesis.py", line 221, in synthesis
    outputs = run_model_torch(
  File "/usr/local/lib/python3.10/dist-packages/TTS/tts/utils/synthesis.py", line 53, in run_model_torch
    outputs = _func(
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    retu

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/trainer/trainer.py", line 1808, in fit
    self._fit()
  File "/usr/local/lib/python3.10/dist-packages/trainer/trainer.py", line 1764, in _fit
    self.test_run()
  File "/usr/local/lib/python3.10/dist-packages/trainer/trainer.py", line 1680, in test_run
    test_outputs = self.model.test_run(self.training_assets)
  File "/usr/local/lib/python3.10/dist-packages/TTS/tts/models/base_tacotron.py", line 157, in test_run
    outputs_dict = synthesis(
  File "/usr/local/lib/python3.10/dist-packages/TTS/tts/utils/synthesis.py", line 221, in synthesis
    outputs = run_model_torch(
  File "/usr/local/lib/python3.10/dist-packages/TTS/tts/utils/synthesis.py", line 53, in run_model_torch
    outputs = _func(
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/TTS/tts/models/tacotro

TypeError: ignored